In [435]:
import os
import sys
import random
import pandas
import numpy
import re

In [535]:
HOME_DIR='./'
PERT_FRAC = 9 # [0.0, 0.5, 1, 4, 9]
OUT_FOLDER = str(PERT_FRAC).replace('.', '_')
ROOT_FOLDER = 'rps' # Don't change this

ACONC = '1.1'
BCONC = '1.0'
CCONC = '0.3'

In [536]:
def original_leak_sections():
    sections = [
        [],
        [],
        [],
        [],
        [
        'reaction [condensed    = 6e-8 /nM/s ] ProduceBCjCk + HelperCCk -> Ck + Cj',
        'reaction [condensed    = 6e-8 /nM/s ] ProduceABrBs + HelperBBs -> Bs + Br',
        'reaction [condensed    = 6e-8 /nM/s ] ProduceCApAq + HelperAAq -> Aq + Ap',
        ]
    ]
    return sections

def shadow_leak_sections():
    sections = [
        [],
        [],
        [],
        [],
        [
           'reaction [condensed    = 6e-8 /nM/s ] shProduceBCjCk + shHelperCCk -> shCk + shCj',
           'reaction [condensed    = 6e-8 /nM/s ] shProduceABrBs + shHelperBBs -> shBs + shBr',
           'reaction [condensed    = 6e-8 /nM/s ] shProduceCApAq + shHelperAAq -> shAq + shAp'
        ]
    ]
    return sections

In [537]:
def read_lines(file):
    with open(file, 'r') as fin:
        lines = fin.readlines()
        lines = [line.strip() for line in lines]
    return lines

def write_lines(lines, file):
    with open(file, 'w') as fout:
        for line in lines:
            fout.write(line)
            fout.write('\n')
            
def eps_ub(d, frac):
    l = [d + d*frac]
    return round(random.choice(l), 9)

def match_replace_rate_const(s, regex='([0-9]\.[0-9]+)', frac=None):
    x = re.search(regex, s)
    if x is not None:
        rep = str(eps_ub(float(x.group()), frac))
        ret = re.sub(regex, rep, s)
        return ret
    else:
        return s

def divide_sections(lines):
    line_numbers = []
    for i, line in enumerate(lines):
        if '#' in line:
            line_numbers.append(i)
    line_numbers.append(len(lines))
    sections = []
    headers = []
    for i in range(len(line_numbers)-1):
        start = line_numbers[i]
        end = line_numbers[i+1]
        sections.append(lines[start+1:end])
        headers.append(lines[start])
    return headers, sections


def merge(ofile, sfile, cfile=None):
    olines = read_lines(ofile)
    slines = read_lines(sfile)
    
    o_heds, o_secs = divide_sections(olines)
    s_heds, s_secs = divide_sections(slines)
    
    o_leak_secs = original_leak_sections() 
    s_leak_secs = shadow_leak_sections()
    
    assert len(o_secs) == len(s_secs)
    assert len(o_secs) == len(o_leak_secs)
    
    headers = o_heds
    sections = [x[0] + x[1] + x[2] + x[3] for x in zip(o_secs, s_secs, o_leak_secs, s_leak_secs)]
   
    
    if cfile is not None:
        clines = read_lines(cfile)
        c_heds, c_secs = divide_sections(clines)
        assert len(c_secs) == len(o_secs)
        sections = [x[0] + list(set(x[1]).difference(set(x[0]))) for x in zip(sections, c_secs)]
    
    return headers, sections

def write_out(headers, sections, out=None, cancel=False):
    lines = []
    for h, s in zip(headers, sections):
        lines += [h]
        lines += s
    subbed_lines = []
#     if cancel: # THIS DOESN't WORK (IDK why). Manually zero out concentrations, lol.
    for line in lines:
        if re.match(r'shCj = hCjR fCR mCR sCR @initial [0-9a-z\.\-]+ nM', line) is not None:
            subbed_lines += [f'shCj = hCjR fCR mCR sCR @initial {CCONC} nM']
            print(line, '\n', subbed_lines[-1])
        elif re.match(r'shCk = hCkR fCR mCR sCR @initial [0-9a-z\.\-]+ nM', line) is not None:
            subbed_lines += [f'shCk = hCkR fCR mCR sCR @initial {CCONC} nM']
            print(line, '\n', subbed_lines[-1])
        elif re.match(r'shAp = hApR fAR mAR sAR @initial [0-9a-z\.\-]+ nM', line) is not None:
            subbed_lines += [f'shAp = hApR fAR mAR sAR @initial {ACONC} nM']
        elif re.match(r'shAq = hAqR fAR mAR sAR @initial [0-9a-z\.\-]+ nM', line) is not None:
            subbed_lines += [f'shAq = hAqR fAR mAR sAR @initial {ACONC} nM']
        elif re.match(r'shBr = hBrR fBR mBR sBR @initial [0-9a-z\.\-]+ nM', line) is not None:
            subbed_lines += [f'shBr = hBrR fBR mBR sBR @initial {BCONC} nM']
        elif re.match(r'shBs = hBsR fBR mBR sBR @initial [0-9a-z\.\-]+ nM', line) is not None:
            subbed_lines += [f'shBs = hBsR fBR mBR sBR @initial {BCONC} nM']
        else:
            subbed_lines.append(line)
    write_lines(subbed_lines, out)
#     else:
#         write_lines(lines, out)

# Add leaks to Original, Shadow, and Merge UnPerturbed

In [538]:
OUT_DIR = os.path.join(HOME_DIR, ROOT_FOLDER, OUT_FOLDER)
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)


In [539]:
ofile = os.path.join(HOME_DIR, ROOT_FOLDER, 'original', 'main_enum.pil')
sfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow', 'main_enum.pil')
cfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'cancel', 'main_enum.pil')
out_nocancel = os.path.join(OUT_DIR, 'orig_shadow_nocancel_enum.pil')
out_cancel = os.path.join(OUT_DIR, 'orig_shadow_cancel_enum.pil')

headers, sections = merge(ofile, sfile)
write_out(headers, sections, out_nocancel)
headers, sections = merge(ofile, sfile, cfile)
write_out(headers, sections, out_cancel, cancel=True)

shCj = hCjR fCR mCR sCR @initial 3 nM 
 shCj = hCjR fCR mCR sCR @initial 0.3 nM
shCk = hCkR fCR mCR sCR @initial 1e-05 nM 
 shCk = hCkR fCR mCR sCR @initial 0.3 nM
shCj = hCjR fCR mCR sCR @initial 3 nM 
 shCj = hCjR fCR mCR sCR @initial 0.3 nM
shCk = hCkR fCR mCR sCR @initial 1e-05 nM 
 shCk = hCkR fCR mCR sCR @initial 0.3 nM


# Perturb

In [540]:
lines = read_lines(os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow', 'main_enum.pil'))
pert_lines = []
for line in lines:
    pert_lines.append(match_replace_rate_const(line, frac=PERT_FRAC))
write_lines(pert_lines, os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow', f'main_pert-{PERT_FRAC}_enum.pil'))

# Add leaks to Original, Shadow, and Perturbed

In [541]:
ofile = os.path.join(HOME_DIR, ROOT_FOLDER, 'original', 'main_enum.pil')
sfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow', f'main_pert-{PERT_FRAC}_enum.pil')
cfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'cancel', 'main_enum.pil')
out_nocancel = os.path.join(OUT_DIR, 'orig_shadow_pert_nocancel_enum.pil') # CHANGE THE NAME IF YOU NEED
out_cancel = os.path.join(OUT_DIR, 'orig_shadow_pert_cancel_enum.pil') # CHANGE THE NAME IF YOU NEED

headers, sections = merge(ofile, sfile)
write_out(headers, sections, out_nocancel) # No cancellation
headers, sections = merge(ofile, sfile, cfile)
write_out(headers, sections, out_cancel, cancel=True) # Cancellation included

shCj = hCjR fCR mCR sCR @initial 3 nM 
 shCj = hCjR fCR mCR sCR @initial 0.3 nM
shCk = hCkR fCR mCR sCR @initial 1e-05 nM 
 shCk = hCkR fCR mCR sCR @initial 0.3 nM
shCj = hCjR fCR mCR sCR @initial 3 nM 
 shCj = hCjR fCR mCR sCR @initial 0.3 nM
shCk = hCkR fCR mCR sCR @initial 1e-05 nM 
 shCk = hCkR fCR mCR sCR @initial 0.3 nM


# Zero Concentration Files

In [542]:
ACONC = '0'
BCONC = '0'
CCONC = '0'

In [543]:
OUT_DIR = os.path.join(HOME_DIR, ROOT_FOLDER, OUT_FOLDER)
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)

ofile = os.path.join(HOME_DIR, ROOT_FOLDER, 'original', 'main_enum.pil')
sfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow', 'main_enum.pil')
cfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'cancel', 'main_enum.pil')
out_nocancel = os.path.join(OUT_DIR, 'orig_shadow_nocancel_enum.pil')
out_cancel = os.path.join(OUT_DIR, 'orig_shadow_cancel_enum.pil')

headers, sections = merge(ofile, sfile)
write_out(headers, sections, out_nocancel)
headers, sections = merge(ofile, sfile, cfile)
write_out(headers, sections, out_cancel, cancel=True)

shCj = hCjR fCR mCR sCR @initial 3 nM 
 shCj = hCjR fCR mCR sCR @initial 0 nM
shCk = hCkR fCR mCR sCR @initial 1e-05 nM 
 shCk = hCkR fCR mCR sCR @initial 0 nM
shCj = hCjR fCR mCR sCR @initial 3 nM 
 shCj = hCjR fCR mCR sCR @initial 0 nM
shCk = hCkR fCR mCR sCR @initial 1e-05 nM 
 shCk = hCkR fCR mCR sCR @initial 0 nM


In [544]:
lines = read_lines(os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow', 'main_enum.pil'))
pert_lines = []
for line in lines:
    pert_lines.append(match_replace_rate_const(line, frac=PERT_FRAC))
write_lines(pert_lines, os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow', f'main_pert-{PERT_FRAC}_zeroconc_enum.pil'))

In [545]:
ofile = os.path.join(HOME_DIR, ROOT_FOLDER, 'original', 'main_enum.pil')
sfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow', f'main_pert-{PERT_FRAC}_zeroconc_enum.pil')
cfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'cancel', 'main_enum.pil')
out_nocancel = os.path.join(OUT_DIR, 'orig_shadow_pert_nocancel_zeroconc_enum.pil') # CHANGE THE NAME IF YOU NEED
out_cancel = os.path.join(OUT_DIR, 'orig_shadow_pert_cancel_zeroconc_enum.pil') # CHANGE THE NAME IF YOU NEED

headers, sections = merge(ofile, sfile)
write_out(headers, sections, out_nocancel) # No cancellation
headers, sections = merge(ofile, sfile, cfile)
write_out(headers, sections, out_cancel, cancel=True) # Cancellation included

shCj = hCjR fCR mCR sCR @initial 3 nM 
 shCj = hCjR fCR mCR sCR @initial 0 nM
shCk = hCkR fCR mCR sCR @initial 1e-05 nM 
 shCk = hCkR fCR mCR sCR @initial 0 nM
shCj = hCjR fCR mCR sCR @initial 3 nM 
 shCj = hCjR fCR mCR sCR @initial 0 nM
shCk = hCkR fCR mCR sCR @initial 1e-05 nM 
 shCk = hCkR fCR mCR sCR @initial 0 nM
